In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Pyomo set-up
!pip install -q pyomo

     |████████████████████████████████| 9.5MB 3.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 256kB 45.3MB/s 
     |████████████████████████████████| 163kB 46.3MB/s 


In [ ]:
# Solver set-up
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

# The Diet Problem
## Summary
The goal of the Diet Problem is to select foods that satisfy daily nutritional requirements at minimum cost. This problem can be formulated as a linear program, for which constraints limit the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet. Danzig (1990) notes that the diet problem was motivated by the US Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet.
## Problem Statement
The Diet Problem can be formulated mathematically as a linear programming problem using the following model.
### Sets
$F$ = set of foods

$N$ = set of nutrients

### Parameters
$c_i$ = cost per serving of food $i$, $ \forall  i \in F$

$a_{ij}$ = amount of nutrient $j$ in food $i$, $\forall i \in F$, $\forall j \in N$

$Nmin_j$ = maximum level of nutrient $j$, $\forall j \in N$

$Nmax_j$ = maximum level of nutrient $j$, $\forall j \in N$

$V_i$ = the volume per serving of food $i$, $\forall i \in F$

$V_max$ = maximum volume of food consumed

### Variables
$x_i$ = number of servings of food $i$ to consume

### Objective
Minimize the total cost of the food $\min \sum_{i \in F} c_i x_i$

### Constraints
Limit nutrient consumption for each nutrient $j \in N$.
$Nmin_j \leq \sum_{i \in F} a_{ij} x_i \leq Nmax_j, \forall j \in N$

Limit the volume of food consumed 
$\sum_{i \in F} V_i x_i \leq Vmax$

Consumption lower bound
$x_i \geq 0, \forall i \in F$

## Pyomo Formulation
We begin by importing the Pyomo package and creating a model object:

In [ ]:
from pyomo.environ import *
infinity = float('inf')

model = AbstractModel()

The sets $F$ and $N$ are declared abstractly using the `Set` component:

In [ ]:
# Foods
model.F = Set()
# Nutrients
model.N = Set()

Similarly, the model parameters are defined abstractly using the `Param` component:

In [ ]:
# Cost of each food
model.c = Param(model.F, within=PositiveReals)
# Amount of nutrient in each food
model.a = Param(model.F, model.N, within=NonNegativeReals)
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# Volume per serving of food
model.V = Param(model.F, within=PositiveReals)
# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)

The `within` option is used in these parameter declarations to define expected properties of the parameters. This information is used to perform error checks on the data that is used to initialize the parameter components.

The `Var` component is used to define the decision variables:

In [ ]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

The `within` option is used to restrict the domain of the decision variables to the non-negative reals. This eliminates the need for explicit bound constraints for variables.

The `Objective` component is used to define the cost objective. This component uses a rule function to construct the objective expression:

In [ ]:
# Minimize the cost of food that is consumed
def cost_rule(model) :
  return sum(model.c[i]*mode.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

Similarly, rule functions are used to define constraint expressions in the `Constraint` component:

In [ ]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j) :
  value = sum(model.a[i,j]*model.x[i] for i in model.F)
  return inequality(model.Nmin[j], value, model.Nmax[j]) # Modified
  
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)

# Limit the volume of food consumed
def volume_rule(model) :
  return sum(mode.V[i]*model.x[i] for i in model.F) <= model.Vmax
model.volume = Constraint(rule=volume_rule)

Putting these declarations all together gives the following model:

In [ ]:
!cat '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet.py'

from pyomo.environ import *
infinity = float('inf')

model = AbstractModel()

# Foods
model.F = Set()
# Nutrients
model.N = Set()

# Cost of each food
model.c    = Param(model.F, within=PositiveReals)
# Amount of nutrient in each food
model.a    = Param(model.F, model.N, within=NonNegativeReals)
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# Volume per serving of food
model.V    = Param(model.F, within=PositiveReals)
# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)

# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

# Minimize the cost of food that is consumed
def cost_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    value = sum(model.a[i,j]*model.x[

## Model Data

Since this is an abstract Pyomo model, the set and parameter values need to be provided to initialize the model. The following data command file provides a synthetic data set:

In [ ]:
!cat '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet.dat'

param:  F:                          c     V  :=
  "Cheeseburger"                 1.84   4.0  
  "Ham Sandwich"                 2.19   7.5  
  "Hamburger"                    1.84   3.5  
  "Fish Sandwich"                1.44   5.0  
  "Chicken Sandwich"             2.29   7.3  
  "Fries"                         .77   2.6  
  "Sausage Biscuit"              1.29   4.1  
  "Lowfat Milk"                   .60   8.0 
  "Orange Juice"                  .72  12.0 ;

param Vmax := 75.0;

param:  N:       Nmin   Nmax :=
        Cal      2000      .
        Carbo     350    375
        Protein    55      .
        VitA      100      .
        VitC      100      .
        Calc      100      .
        Iron      100      . ;

param a:
                               Cal  Carbo Protein   VitA   VitC  Calc  Iron :=
  "Cheeseburger"               510     34     28     15      6    30    20
  "Ham Sandwich"               370     35     24     15     10    20    20
  "Hamburger"                  500     42

Set data is defined with the `set` command, and parameter data is defined with the `param` command.

This data set considers the problem of designing a daily diet with only food from a fast food chain.

## Solution
Pyomo includes a `pyomo` command that automates the construction and optimization of models. The GLPK solver can be used in this simple example:

In [ ]:
!pyomo solve --solver=glpk  '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet.py'  '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet.dat'

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.27] Creating model
[    0.30] Applying solver
[    0.32] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 15.05
    Solver results file: results.json
[    0.32] Applying Pyomo postprocessing actions
[    0.32] Pyomo Finished


In [ ]:
!mv results.json '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet_results.json'
!cat '/content/drive/MyDrive/Optimizacion_Convexa/01_Warm_up/The Diet Problem/diet_results.json'

{
    "Problem": [
        {
            "Lower bound": 15.05,
            "Name": "unknown",
            "Number of constraints": 10,
            "Number of nonzeros": 77,
            "Number of objectives": 1,
            "Number of variables": 10,
            "Sense": "minimize",
            "Upper bound": 15.05
        }
    ],
    "Solution": [
        {
            "number of solutions": 1,
            "number of solutions displayed": 1
        },
        {
            "Constraint": "No values",
            "Gap": 0.0,
            "Message": null,
            "Objective": {
                "cost": {
                    "Value": 15.05
                }
            },
            "Problem": {},
            "Status": "optimal",
            "Variable": {
                "x[Cheeseburger]": {
                    "Value": 4.0
                },
                "x[Fish Sandwich]": {
                    "Value": 1.0
                },
                "x[Fries]": {
                    "Val

This solution shows that for about $15 per day, a person can get by with 4 cheeseburgers, 5 fries, 1 fish sandwich and 4 milks.

## References
* G.B. Dantzig. The Diet Problem, Interfaces 20(4), 1990, 43-47